# Big data made small

Follow along the notebook to learn how to use SQL through [DuckDB](https://DuckDB.org/), a **fast** database system, available through both python and R.

In [1]:
# Import today's main character
import DuckDB

## Basic building blocks of SQL querying

SQL is a declarative language that allows you to query tables from databases. There's plenty of database ecosystems out there. If you work with data in a large organization, you'll encounter at least one. Often more than one.

DuckDB is an **analytical** database system that allows extreme flexibility in querying data in different formats. Today we will focus primarily on `parquet` files (though you could also directly work on e.g. `CSV`s).

[`parquet`](https://parquet.apache.org/) is a data format optimized for efficient data retrieval. There's more and more data available as parquet files out there, and there's multiple way to transform your data into parquet - the easiest probably being [exporting from pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_parquet.html). 

For today, we'll assume you got the data in this format, which will be created in your local `./data/` folder as soon as you run `uv run create_data.py`.

This introduction runs through the basic syntax of `SELECT` statements in SQL. These constitutes the vast majority of SQL work for an analyst.

### SELECT FROM + LIMIT

`SELECT` statements are structured as follows

![image](../imgs/select.png)

A basic query could then look something like

```SQL
SELECT
    column1, column2
FROM thistable
LIMIT 5
```

This query means "Extract the first 5 records of columns `column1` and `column2` from table `thistable`.

You can also usethe character `*` as jolly to return all columns in a table.

> **WARNING**
> 
> If you are doing exploratory analysis "just to look at the data", **always** impose a `LIMIT` condition to ensure you don't load everything in the memory. DuckDB (and database engines in general) can handle larger-than-RAM datasets. Loading a huge table  up is a safe recipe for crashing your computer sooner or later.

DuckDB allows to run SQL queries **directly on your `parquet` files**! This means you will not load the whole dataset in memory. DuckDB will intelligently scan the data, select only the tidbits you need, alaborate them, and return them.

So say we want to "look at what's in `../data/tests.parquet` file. We can adapt the basic query above and transform it into  

```SQL
select te.*
from read_parquet('../data/tests.parquet') as te
limit 5
```

Where `read_parquet()` is a DuckDB function reading parquet files.

> **NOTE**
>
> Strings in SQL always are delimited by character `'`, **not** `"`


With DuckDB, we can run SQL query simply by encapsulating them into `DuckDB.sql()` as follows 

In [5]:
%%time
DuckDB.sql(
    """
    select te.*
    from read_parquet('../data/tests.parquet') as te
    limit 5
    """
)

CPU times: user 2 ms, sys: 453 µs, total: 2.46 ms
Wall time: 2.17 ms


┌─────────────────────┬─────────┬──────────┐
│      test_time      │   id    │ positive │
│    timestamp_ns     │  int64  │  int64   │
├─────────────────────┼─────────┼──────────┤
│ 2022-02-11 00:28:50 │ 2004206 │        0 │
│ 2022-03-20 10:41:34 │  113107 │        1 │
│ 2022-08-17 03:59:32 │  979034 │        0 │
│ 2022-09-08 06:24:04 │ 2765675 │        0 │
│ 2022-12-13 19:01:15 │ 2070417 │        0 │
└─────────────────────┴─────────┴──────────┘

Easy! We had a peek at the data in a tiny fraction of a second, without the need of loading all the dataset in the memory.

`DuckDB.sql()` returns a relation object. You can convert it in other formats, like a `pandas.DataFrame`, e.g.

In [7]:
%%time
df = DuckDB.sql(
    """
    select te.*
    from read_parquet('../data/tests.parquet') as te
    limit 5
    """
).df()

df

CPU times: user 21.1 ms, sys: 0 ns, total: 21.1 ms
Wall time: 11.5 ms


,test_time,id,positive
0,2022-02-11 00:28:50,2004206,0
1,2022-03-20 10:41:34,113107,1
2,2022-08-17 03:59:32,979034,0
3,2022-09-08 06:24:04,2765675,0
4,2022-12-13 19:01:15,2070417,0


### WHERE and ORDER BY

We can add a condition to the query. For example, we can get all tests done by `id=5` after August 2022 by using a `WHERE` statement.

Note that now I am not using a `LIMIT` condition anymore, as I know the data output will be limited in this case.

I am also using `ORDER BY` at the end to order rows chronologically.

In [8]:
%%time
DuckDB.sql(
    """
    select te.*
    from read_parquet('../data/tests.parquet') as te
    where te.id=5
        and cast(te.test_time as date)>='2022-08-01'
    ORDER BY te.test_time
    """
)

CPU times: user 2.15 ms, sys: 1.13 ms, total: 3.29 ms
Wall time: 2.63 ms


┌─────────────────────┬───────┬──────────┐
│      test_time      │  id   │ positive │
│    timestamp_ns     │ int64 │  int64   │
├─────────────────────┼───────┼──────────┤
│ 2022-08-13 01:20:47 │     5 │        0 │
│ 2022-08-31 09:26:49 │     5 │        0 │
│ 2022-09-10 07:38:21 │     5 │        0 │
│ 2022-11-12 11:50:07 │     5 │        0 │
└─────────────────────┴───────┴──────────┘

### GROUP BY

`GROUP BY` serves to aggregate our data. Via SQL, you can aggregate the data *without* loading all rows in a pandas dataframe. For example, say we want to count how many tests have been done by month in the `tests` data, and the percentage of them showing up positive (here data is simulated, so no useful pattern will show up).

Here's how you could do it.

In [9]:
%%time
DuckDB.sql(
    """
    select
        month(test_time) as month
        , COUNT(*) as n_tests
        , MEAN(positive) as share_positive
    from read_parquet('../data/tests.parquet') as te
    GROUP BY month(test_time)
    ORDER BY month(test_time)
    """
)

CPU times: user 2.9 ms, sys: 0 ns, total: 2.9 ms
Wall time: 3.79 ms


┌───────┬─────────┬─────────────────────┐
│ month │ n_tests │   share_positive    │
│ int64 │  int64  │       double        │
├───────┼─────────┼─────────────────────┤
│     1 │ 3834859 │ 0.07970618997986627 │
│     2 │ 3459620 │ 0.07996514068019031 │
│     3 │ 3829567 │ 0.07966592567775939 │
│     4 │ 3710265 │ 0.07995116251804116 │
│     5 │ 3835185 │ 0.07991817865370249 │
│     6 │ 3705759 │ 0.08002328267974253 │
│     7 │ 3833010 │ 0.07986386677832825 │
│     8 │ 3830474 │ 0.08007520740253034 │
│     9 │ 3708918 │ 0.07996024716642428 │
│    10 │ 3835433 │  0.0799581168540814 │
│    11 │ 3708389 │ 0.07998702401501029 │
│    12 │ 3708521 │ 0.08023252396305697 │
├───────┴─────────┴─────────────────────┤
│ 12 rows                     3 columns │
└───────────────────────────────────────┘

## Expanding `FROM`

### Subqueries and Common Table Expressions (CTEs)

Let's go back to `FROM`. The easiest way of using `FROM` is to use as input a single table. But a table can also be the result of a query itself!

For example, let's see how the distribution of many times each person in the dataset was tested. To do so, we need to

1. calculate the number of tests per id in the data
2. sum how many ID occurrences per number of tests. 

Or:

In [11]:
%%time
DuckDB.sql(
    """
    SELECT n_tests, count(id) as n_ppl 
    FROM (
        SELECT id, count(id) as n_tests 
        from read_parquet('../data/tests.parquet') 
        GROUP BY id
    ) as t
    GROUP BY n_tests
    ORDER BY n_tests
    """
)

CPU times: user 1.96 ms, sys: 731 µs, total: 2.7 ms
Wall time: 3.42 ms


┌─────────┬────────┐
│ n_tests │ n_ppl  │
│  int64  │ int64  │
├─────────┼────────┤
│       1 │     12 │
│       2 │    101 │
│       3 │    515 │
│       4 │   1927 │
│       5 │   5826 │
│       6 │  14424 │
│       7 │  30894 │
│       8 │  57953 │
│       9 │  97722 │
│      10 │ 145706 │
│       · │     ·  │
│       · │     ·  │
│       · │     ·  │
│      29 │   1240 │
│      30 │    680 │
│      31 │    359 │
│      32 │    136 │
│      33 │     57 │
│      34 │     25 │
│      35 │     11 │
│      36 │      3 │
│      37 │      1 │
│      39 │      1 │
├─────────┴────────┤
│     38 rows      │
│    (20 shown)    │
└──────────────────┘

The part in parentheses after `FROM` is a subquery. The problem with subqueries is that, even with copious indentation, queries quickly become hard to read and debug.

Enter **Common Table Expressions (CTEs)**. They are almost the same as subqueries, but we defined them *at the beginning of a query*, encapsulating them into a `WITH ctename as (CTE)` statement, making the query gets more legible.

Another advantage of CTEs is that we'd be able to reuse table `t` in the example below multiple times in a query, if needed, without rewriting the subquery multiple times. As such, CTEs serve as temporary tables.

In [12]:
%%time
DuckDB.sql(
    """
    WITH t as (
        SELECT id, count(id) as n_tests 
        from read_parquet('../data/tests.parquet') 
        GROUP BY id
    )
    SELECT n_tests, count(id) as n_ppl 
    FROM t
    GROUP BY n_tests
    ORDER BY n_tests
    """
)

CPU times: user 0 ns, sys: 3.1 ms, total: 3.1 ms
Wall time: 2.03 ms


┌─────────┬────────┐
│ n_tests │ n_ppl  │
│  int64  │ int64  │
├─────────┼────────┤
│       1 │     12 │
│       2 │    101 │
│       3 │    515 │
│       4 │   1927 │
│       5 │   5826 │
│       6 │  14424 │
│       7 │  30894 │
│       8 │  57953 │
│       9 │  97722 │
│      10 │ 145706 │
│       · │     ·  │
│       · │     ·  │
│       · │     ·  │
│      29 │   1240 │
│      30 │    680 │
│      31 │    359 │
│      32 │    136 │
│      33 │     57 │
│      34 │     25 │
│      35 │     11 │
│      36 │      3 │
│      37 │      1 │
│      39 │      1 │
├─────────┴────────┤
│     38 rows      │
│    (20 shown)    │
└──────────────────┘

### The real deal: JOINs

As mentioned in the slides, working with relational databases means joining lots of table together. That's the real superpower of SQL.

`JOIN` is how we combine multiple tables. Essentially that'd be `pd.DataFrame.merge()` - just exceptionally more powerful.

There are different types of `JOIN`s. The table below summarizes the key differences, and I will explain them briefly at the workshop, but as analysts you'd most often deal with two of these: `INNER JOIN` and `LEFT JOIN`.

![image](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.techagilist.com%2Fwp-content%2Fuploads%2F2018%2F07%2Fsql-joins.png&f=1&nofb=1&ipt=187475e01030755dedb9dcd23da61bdb8650b747c4fca8004eb442d243191213&ipo=images)

The idea is simply to take all rows from a table, and attach all rows from a second table according to a (set of) column(s). So for example I want to join each id from Table A to each ID of Table B.

* `INNER JOIN`: Returns *only rows where IDs are found in **both** tables*
* `LEFT JOIN`: Returns *all rows from table A*, joining data from Table B when available (the rest being `NULL`)

In the simulated data, for example, let's say that the `tests` data contains tests for all the population, while the `transactions` data only contains transaction of clients of a specific financial institute. Indeed, we could count how many `DISTINCT` IDs we have in both tables.

In [8]:
%%time
print(
    DuckDB.sql(
        """select count(distinct id) as id_tests
        from read_parquet('../data/tests.parquet')
        """
    )
)
print(
    DuckDB.sql(
        """
        select count(distinct id) as id_transactions
        from read_parquet('../data/transactions.parquet')
        """
    )
)

┌──────────┐
│ id_tests │
│  int64   │
├──────────┤
│  2999999 │
└──────────┘

┌─────────────────┐
│ id_transactions │
│      int64      │
├─────────────────┤
│          899999 │
└─────────────────┘

CPU times: user 24.2 s, sys: 1.63 s, total: 25.9 s
Wall time: 3.65 s


Let's say we want to create a dataframe with as columns:
- The ID of the person (key)
- Whether a person ever tasted positive
- The number of transactions the person made

To do so, first we aggregate the tests and transaction table, then we join them.

Let's start with a `LEFT JOIN`, where the `LEFT` table is the aggregated test table.

We get out 2999999 rows (the number of unique IDs in the `tests` table), but many of them have a missing `n_transactions`. We do not have transaction data for those individuals. However, we still retain the full sample, as we **do** have test data for the full population.

In [13]:
%%time
DuckDB.sql(
    """
    with te as (
        SELECT id, max(positive) as ever_positive
        FROM read_parquet('../data/tests.parquet')
        GROUP BY id
    ), tr as (
        SELECT id, count(id) as n_transactions
        FROM read_parquet('../data/transactions.parquet')
        GROUP BY id
    )
    SELECT
        te.*, tr.n_transactions
    FROM te
        LEFT JOIN tr
            ON te.id=tr.id
    ORDER BY te.id DESC
    """
).df()

CPU times: user 36.7 s, sys: 3.29 s, total: 40 s
Wall time: 4.98 s


,id,ever_positive,n_transactions
0,2999999,1,NaN
1,2999998,1,NaN
2,2999997,1,NaN
3,2999996,1,NaN
4,2999995,1,NaN
...,...,...,...
2999994,5,0,121.0
2999995,4,1,113.0
2999996,3,1,113.0
2999997,2,1,92.0


Were we to do an `INNER JOIN` instead, we'd get 899999 rows - the clients having *both* test and transaction data.

In [14]:
%%time
DuckDB.sql(
    """
    with te as (
        SELECT id, max(positive) as ever_positive
        FROM read_parquet('../data/tests.parquet')
        GROUP BY id
    ), tr as (
        SELECT id, count(id) as n_transactions
        FROM read_parquet('../data/transactions.parquet')
        GROUP BY id
    )
    SELECT
        count(*)
    FROM te
    INNER JOIN tr
        ON te.id=tr.id
    """
)

CPU times: user 3.61 ms, sys: 436 µs, total: 4.04 ms
Wall time: 3.13 ms


┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       899999 │
└──────────────┘

### Non-trivial JOINs

The joins we have seen above are quite straightfoward: One key, with equality. These joins are also straighforward with e.g. pandas. However, SQL allows you to handle much more complex joins.

For example, joins on 
- Multiple keys (e.g. id and date)
- Functions of keys (e.g. month of a date)
- Unequal joins (e.g. on X>Y)

To try these out, let's try to find out all transactions made by people who tested positive 2 to 5 days before the positive test: That is, when they were asymptomatic but likely contagious.

Now, this starts to be a somewhat complex operation. We need to:

* Extract all positive tests. That's 3.597.337 positive tests in the simulated data.
* Find all transactions made by the individual being tested. The number of rows would grow to 108.989.386 transactions.
* Only select those where the time range fits.

The code belows does this join in `pandas`. It's uncommented, as you should only try it out if you have ~10GB RAM memory to spare. It takes about 35 seconds to run on my machine.

> **Note:** There'll be a few records not matching due to how pandas and sql engines treat date boundaries.

In [11]:
# %%time
# import pandas as pd
# tests = pd.read_parquet('../data/tests.parquet')
# transactions = pd.read_parquet('../data/transactions.parquet')

# pt = tests.loc[tests["positive"]==1]
# tt = pt.merge(transactions, on='id', how='inner')
# tt["infector_days_to_test"] = (tt["transaction_time"] - tt["test_time"]).dt.days + 1
# tt = tt[tt["infector_days_to_test"].between(-5, -2)]
# tt

The same query in in DuckDB runs in ~3 seconds on my machine, and the RAM is not nearly as challenged.

In [15]:
%%time
experiments = DuckDB.sql(
    """
    select t.id as infector_id, t.test_time as infector_test_time
        , tt.transaction_time as infector_transaction_time, tt.shop
        , date_sub('days', t.test_time, tt.transaction_time) as infector_days_to_test
        --, row_number() over() as experiment_number
    from read_parquet('../data/tests.parquet') as t
    inner join read_parquet('../data/transactions.parquet') as tt
        on tt.id=t.id 
        and date_sub('days', t.test_time, tt.transaction_time) between -5 and -2
    where t.positive=1
    --order by t.id, infector_test_date, infector_transaction_time
    """
).df()
experiments

CPU times: user 35.3 s, sys: 1.4 s, total: 36.7 s
Wall time: 6.08 s


,infector_id,infector_test_time,infector_transaction_time,shop,infector_days_to_test
0,573339,2022-03-22 00:28:47,2022-03-16 10:45:35,39692,-5
1,503826,2022-10-22 12:30:17,2022-10-16 19:49:41,9899,-5
2,50017,2022-08-17 08:53:12,2022-08-13 09:24:55,9356,-3
3,55963,2022-12-11 20:43:58,2022-12-08 14:15:47,22735,-3
4,273094,2022-07-12 23:04:20,2022-07-07 14:47:12,638,-5
...,...,...,...,...,...
1184794,176454,2022-09-30 11:16:31,2022-09-25 09:20:43,49721,-5
1184795,317251,2022-01-27 06:37:44,2022-01-24 12:49:30,1600,-2
1184796,674914,2022-04-08 15:22:07,2022-04-04 17:30:34,27595,-3
1184797,142380,2022-11-05 02:43:09,2022-11-01 21:18:17,14394,-3


## All together: Find all transactions by people crossing an infector in a store!

Now the final step. Let's take the *experiments* we identified in the previous query, and join them with all transactions by other people happening in the same store within 30 minutes of the original transactions. Piece of cake!

>The way I'd approach building the query:
>1) Start from the basic ingredients, i.e. the tables. We have a query creating our "experiments". Start from that.
>2) Understand which data I need additionally - here the `transactions` data. Let's `JOIN` it.
>3) Write down the conditions under which combinations of transactions are acceptable. For example, they need to be
>    * In the same shop
>    * Not by the same person (the "*infector*")
>    * Within a specific time range
>4) Potentially construct additional variables (here "treatment")

In [13]:
%%time
DuckDB.sql(
    """
    with experiments as (
        select t.id as infector_id, t.test_time as infector_test_time
            , tt.transaction_time as infector_transaction_time, tt.shop
            , date_sub('days', t.test_time, tt.transaction_time) as infector_days_to_test
            , row_number() over() as experiment_number
        from read_parquet('../data/tests.parquet') as t
        inner join read_parquet('../data/transactions.parquet') as tt
            on tt.id=t.id 
            and date_sub('days', t.test_time, tt.transaction_time) between -5 and -2
        where t.positive=1
        -- a small number to test it out
        limit 1000
    ) 
    select e.*
    , t2.id as exposed_id, t2.transaction_time as exposed_transaction_time
    , date_sub('minutes', t2.transaction_time, e.infector_transaction_time) as minutes_apart
    , case when date_sub('minutes', t2.transaction_time, e.infector_transaction_time) between -5 and +5 then 1 
        when date_sub('minutes', t2.transaction_time, e.infector_transaction_time) between -30 and -16 then 0
        else NULL end as treat
    from experiments as e
    inner join read_parquet('../data/transactions.parquet') as t2
        -- in the same shop
        on t2.shop=e.shop
        -- I do not want the infector transactions there!
        and t2.id!=e.infector_id
        --order by experiment_number, exposed_transaction_time
        and t2.transaction_time between
            e.infector_transaction_time - interval 30 minutes
            and e.infector_transaction_time + interval 30 minutes
    """
).df()

CPU times: user 9.97 s, sys: 621 ms, total: 10.6 s
Wall time: 1 s


,infector_id,infector_test_time,infector_transaction_time,shop,infector_days_to_test,experiment_number,exposed_id,exposed_transaction_time,minutes_apart,treat
0,500782,2022-01-19 11:19:20,2022-01-14 03:52:43,18126,-5,453,90249,2022-01-14 04:09:09,-16,0.0
1,148618,2022-02-24 02:25:01,2022-02-19 21:02:56,198,-4,456,14665,2022-02-19 20:45:07,17,NaN
2,506009,2022-03-16 09:29:02,2022-03-11 02:59:09,33381,-5,575,626755,2022-03-11 02:47:37,11,NaN
3,801792,2022-01-14 12:38:35,2022-01-11 12:26:23,3837,-3,557,500553,2022-01-11 12:46:55,-20,0.0
4,767998,2022-08-11 06:03:00,2022-08-08 15:24:36,39587,-2,57,282471,2022-08-08 15:32:42,-8,NaN
...,...,...,...,...,...,...,...,...,...,...
235,863731,2022-04-04 12:03:57,2022-04-01 05:29:46,31105,-3,977,864781,2022-04-01 05:59:17,-29,0.0
236,73403,2022-04-06 20:13:09,2022-04-03 04:21:20,9184,-3,524,289682,2022-04-03 04:43:51,-22,0.0
237,824790,2022-07-22 06:12:06,2022-07-18 19:41:32,35811,-3,425,585233,2022-07-18 19:36:04,5,1.0
238,802447,2022-02-09 14:17:31,2022-02-03 18:02:14,17825,-5,410,113215,2022-02-03 17:55:18,6,NaN


### Using dataframes in memory in your queries

One of the blackest of black magics of DuckDB when you are using it through a python API is that you can directly refer in the queries to existing `pandas.DataFrame`s in memory.

For example, in the code above we saved the results of our `experiments` query in a specific dataframe.

In [17]:
experiments.head()

,infector_id,infector_test_time,infector_transaction_time,shop,infector_days_to_test
0,573339,2022-03-22 00:28:47,2022-03-16 10:45:35,39692,-5
1,503826,2022-10-22 12:30:17,2022-10-16 19:49:41,9899,-5
2,50017,2022-08-17 08:53:12,2022-08-13 09:24:55,9356,-3
3,55963,2022-12-11 20:43:58,2022-12-08 14:15:47,22735,-3
4,273094,2022-07-12 23:04:20,2022-07-07 14:47:12,638,-5


Here we can just reference it in the query below. There is no difference in perfomance here - but it can be an extremely useful tool when working with data coming from different sources.

In [18]:
%%time
DuckDB.sql(
    """
    select e.*
    , t2.id as exposed_id, t2.transaction_time as exposed_transaction_time
    , date_sub('minutes', t2.transaction_time, e.infector_transaction_time) as minutes_apart
    , case when date_sub('minutes', t2.transaction_time, e.infector_transaction_time) between -5 and +5 then 1 
        when date_sub('minutes', t2.transaction_time, e.infector_transaction_time) between -30 and -16 then 0
        else NULL end as treat
    from experiments as e
    inner join read_parquet('../data/transactions.parquet') as t2
        -- in the same shop
        on t2.shop=e.shop
        -- I do not want the infector transactions there!
        and t2.id!=e.infector_id
        --order by experiment_number, exposed_transaction_time
        and t2.transaction_time between
            e.infector_transaction_time - interval 30 minutes
            and e.infector_transaction_time + interval 30 minutes
    limit 100
    """
).df()

CPU times: user 35.8 s, sys: 593 ms, total: 36.4 s
Wall time: 4.29 s


,infector_id,infector_test_time,infector_transaction_time,shop,infector_days_to_test,exposed_id,exposed_transaction_time,minutes_apart,treat
0,49360,2022-01-11 22:55:15,2022-01-06 23:25:53,11019,-4,435581,2022-01-06 23:19:11,6,NaN
1,659851,2022-11-29 13:38:06,2022-11-26 13:50:25,36673,-2,863791,2022-11-26 14:03:42,-13,NaN
2,40490,2022-05-31 09:18:07,2022-05-25 10:02:48,32017,-5,492819,2022-05-25 09:50:51,11,NaN
3,442295,2022-10-30 23:14:44,2022-10-25 21:38:03,29922,-5,893871,2022-10-25 21:16:52,21,NaN
4,554211,2022-03-28 21:14:51,2022-03-23 00:32:54,12024,-5,486645,2022-03-23 00:09:39,23,NaN
...,...,...,...,...,...,...,...,...,...
95,686763,2022-04-15 10:53:29,2022-04-12 03:34:01,13145,-3,327692,2022-04-12 03:12:56,21,NaN
96,318466,2022-12-06 20:12:46,2022-12-01 02:24:05,49734,-5,12970,2022-12-01 02:35:07,-11,NaN
97,692908,2022-06-28 09:28:36,2022-06-23 00:51:41,30023,-5,700710,2022-06-23 00:42:29,9,NaN
98,814458,2022-08-17 09:23:47,2022-08-14 18:00:36,25449,-2,309604,2022-08-14 18:20:52,-20,0.0


### Data versioning becoming code versioning

Putting your queries in notebooks is all fine and well for quick analysis. However, among their many problems, notebooks famously do not agree much with proper code versioning.

However, we could easily save our queries in separate files, and refer to the in our analytical products. That way, the **queries** themselves can be versioned. And by properly using version control, it allows you to essentially version control your data if your raw data sources are immutable.

For fun, the example below shows you also how to parametrize your query.

In [25]:
def read_file(filename):
    try:
        with open(filename, 'r') as file:
            content = file.read()
            return content
    except FileNotFoundError:
        print(f"File {filename} not found.")
        return None

DuckDB.execute(read_file("myquery.sql"), parameters={'nlimit': 100}).df()

,infector_id,infector_test_time,infector_transaction_time,shop,infector_days_to_test,experiment_number,exposed_id,exposed_transaction_time,minutes_apart,treat
0,767998,2022-08-11 06:03:00,2022-08-08 15:24:36,39587,-2,58,282471,2022-08-08 15:32:42,-8,NaN
1,435335,2022-06-25 13:30:14,2022-06-21 15:39:22,5567,-3,88,211900,2022-06-21 15:31:35,7,NaN
2,95721,2022-02-05 09:19:02,2022-01-31 04:58:23,22520,-5,53,294473,2022-01-31 04:37:20,21,NaN
3,193004,2022-10-03 14:08:05,2022-09-28 15:37:58,39911,-4,40,76952,2022-09-28 15:58:28,-20,0.0
4,220515,2022-04-13 16:12:43,2022-04-11 06:22:28,19989,-2,84,118169,2022-04-11 06:40:30,-18,0.0
5,201825,2022-11-28 10:45:39,2022-11-25 08:45:51,44445,-3,72,659229,2022-11-25 08:43:36,2,1.0
6,226689,2022-01-18 05:35:19,2022-01-14 12:26:48,12821,-3,46,883995,2022-01-14 12:09:16,17,NaN
7,435335,2022-06-25 13:30:14,2022-06-21 15:39:22,5567,-3,88,91756,2022-06-21 16:02:17,-22,0.0
8,811227,2022-12-03 00:55:12,2022-11-28 03:28:56,12290,-4,23,456939,2022-11-28 03:23:14,5,1.0
9,337433,2022-12-22 19:35:48,2022-12-19 05:23:20,25095,-3,11,522449,2022-12-19 05:11:52,11,NaN
